In [72]:
import pandas as pd
import os
from datetime import datetime

In [32]:
# Lê o de-para

depara_atividades = pd.read_excel("DE-PARA.xlsx", sheet_name='ATIVIDADES')
depara_materiais = pd.read_excel("DE-PARA.xlsx", sheet_name='MATERIAIS')
depara_materiais = depara_materiais.loc[depara_materiais['STATUS'] == 'FORA DE PADRÃO']

In [33]:
# Lê o caderno de serviços

caderno_servicos = pd.read_excel("caderno de serviços.xlsx")

In [34]:
# Lê detalhamento dos materiais

detalhamento_materiais = pd.read_excel("Detalhamento dos materiais.xlsx")

In [35]:
# Lê a ZPS60

try:
    zps60_inst = pd.read_fwf('_Mat_Serv_a_Instalar.txt', encoding = 'unicode_escape', colspecs = [(0, 24), (24, 60), (60, 140), (140, 143), (143, 154), (154, 164), (164, -1)], names = ['projeto', 'codigo', 'descricao', 'und', 'qnt', 'valor', 'tipo'])
    zps60_inst['projeto'] = zps60_inst['projeto'].replace(to_replace=r'B', value='', regex=True)
    zps60_inst['projeto'] = zps60_inst['projeto'].replace(to_replace=r'X', value='', regex=True)
    zps60_inst['projeto'] = zps60_inst['projeto'].replace(to_replace=r'Y', value='', regex=True)
    zps60_inst['projeto'] = zps60_inst['projeto'].astype(int)
except:
    zps60_inst = ''

try:
    zps60_dest = pd.read_fwf('_Mat_Serv_a_Desativar.txt', encoding = 'unicode_escape', colspecs = [(0, 24), (24, 60), (60, 140), (140, 143), (143, 154), (154, 164), (164, -1)], names = ['projeto', 'codigo', 'descricao', 'und', 'qnt', 'valor', 'tipo'])
    zps60_dest['projeto'] = zps60_dest['projeto'].replace(to_replace=r'B', value='', regex=True)
    zps60_dest['projeto'] = zps60_dest['projeto'].replace(to_replace=r'X', value='', regex=True)
    zps60_dest['projeto'] = zps60_dest['projeto'].replace(to_replace=r'Y', value='', regex=True)
    zps60_dest['projeto'] = zps60_dest['projeto'].astype(int)
except:
    zps60_dest = ''


In [36]:
# Separa materiais das atividades

if ((type(zps60_inst) != str) and (type(zps60_dest) != str)):
    atividades_inst = zps60_inst.loc[zps60_inst['tipo'] == 2]
    atividades_dest = zps60_dest.loc[zps60_dest['tipo'] == 2]
    atividades = pd.concat([atividades_inst, atividades_dest])
    atividades.reset_index(inplace=True, drop=True)
elif (type(zps60_inst) != str):
    atividades_inst = zps60_inst.loc[zps60_inst['tipo'] == 2]
    atividades = atividades_inst
elif (type(zps60_dest) != str):
    atividades_dest = zps60_dest.loc[zps60_dest['tipo'] == 2]
    atividades = atividades_dest

materiais = zps60_inst.loc[zps60_inst['tipo'] == 1]
materiais['codigo'] = materiais['codigo'].astype(int)
materiais.reset_index(inplace=True, drop=True)

C:\Users\PC1210\AppData\Local\Temp\ipykernel_20176\4039113865.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  materiais['codigo'] = materiais['codigo'].astype(int)


In [37]:
# Define valores do fator K

fator_k = pd.DataFrame([['LV', 2.7],['REDES', 1.68]], columns=['GRUPO','FATOR_K'])

In [38]:
####### TRATANDO ATIVIDADES

for count, value in enumerate(atividades.values):
    
    if value[1] in depara_atividades['cod_de'].values:
        linha = depara_atividades.loc[depara_atividades['cod_de'] == value[1]].values
        atividades.loc[count, 'codigo'] = linha[0][2]
        atividades.loc[count, 'descricao'] = linha[0][3]

    if value[2] == 'INSTALAR POSTE 9 A 14 METROS' or value[2] == 'INSTALAR POSTE 14 METROS OU SUPERIOR' or value[2] == 'INSTALAR POSTE 7 METROS' or value[2] == 'INST POSTE 10 A 16 METROS' or value[2] == 'INST POSTE 7 A 9 METROS':
        atividades = pd.concat([atividades, pd.DataFrame([[value[0], 'SIR0000001', 'CAVA NORMAL EM SOLO COMUM', 'UND', value[4], value[4]*138.7050026, 2]], columns=['projeto', 'codigo', 'descricao', 'und', 'qnt', 'valor', 'tipo'])])


atividades = atividades.dropna()
atividades = atividades.loc[~atividades['codigo'].str.contains(r'SDMT', regex=True)] # Transporte
atividades = atividades.loc[~atividades['codigo'].str.contains(r'SDET', regex=True)] # Transporte
atividades = atividades.loc[atividades['codigo'] != 'SDMMU4209SI'] # Deslocamento de turma pesada
atividades = atividades.loc[atividades['codigo'] != 'SDEMU1411II'] # Instalar ramal mono
atividades = atividades.loc[atividades['codigo'] != 'SDEMU1412II'] # Instalar ramal poli
atividades = atividades.loc[atividades['descricao'] != 'INSTALAR PLACA DA OBRA']
atividades = atividades.groupby(['projeto', 'codigo', 'descricao', 'und', 'tipo'], as_index=False).sum()

In [39]:
####### TRATANDO MATERIAIS

for count, value in enumerate(materiais['codigo'].values):
    if value in depara_materiais['DE'].values:
        linha = depara_materiais.loc[depara_materiais['DE'] == value].values
        materiais.loc[count, 'codigo'] = linha[0][3]


In [40]:
# Junta materiais e atividades

orcamentos = pd.concat([atividades,materiais])
orcamentos.reset_index(inplace=True, drop=True)

In [41]:
# Exclui projetos com código antigo

excluir = pd.concat([
    atividades.loc[atividades['codigo'].str.contains(r'BC', regex=True)],
    atividades.loc[atividades['codigo'].str.contains(r'BS', regex=True)],
    atividades.loc[atividades['codigo'].str.contains(r'BT', regex=True)]
])


excluir = excluir.groupby('projeto', as_index=False).sum()
excluir = excluir['projeto'].values

for i in excluir:
    orcamentos = (orcamentos.loc[orcamentos['projeto'] != i])

In [42]:
# Adiciona coluna de etapas

etapa = []
for count, value in enumerate(orcamentos['codigo'].values):
    if value in caderno_servicos['codigo'].values:
        linha = caderno_servicos.loc[caderno_servicos['codigo'] == value].values
        etapa.append(linha[0][5])
    else:
        etapa.append('')

orcamentos.insert(1,'etapa', etapa)

orcamentos = orcamentos.loc[(orcamentos['etapa'] != '') | (orcamentos['tipo'] == 1)]

Elaboração da planilha de diagnóstico

In [43]:
# Adiciona coluna com valor de mão de obra dos projetos

mao_de_obra = orcamentos.loc[orcamentos['tipo'] == 2]
mao_de_obra = mao_de_obra.merge(caderno_servicos[['codigo','Vlr Referência Único', 'GRUPO']], on=['codigo'], how='inner')
mao_de_obra = mao_de_obra.merge(fator_k)
mao_de_obra['Valor Total'] = mao_de_obra['qnt']*mao_de_obra['Vlr Referência Único']*mao_de_obra['FATOR_K']
mao_de_obra = mao_de_obra.groupby(['projeto'], as_index=False).sum()
mao_de_obra = mao_de_obra[['projeto', 'Valor Total']]

In [44]:
orcamento_material

,projeto,etapa,codigo,descricao,und,tipo,qnt,valor,CATEGORIA,DETALHE,NV TENSÃO
0,152603,,3420090,MANILHA SAPATILHA ACO 5000DAN,CDA,1,44.0,343.64,FERRAGEM,NaN,NaN
79,152603,,2412003,"CONETOR PERF 10,0- 35,0/ 1,5- 6,0",CDA,1,5.0,13.10,FERRAGEM,NaN,NaN
109,152603,,2660002,"FITA ISOL PVC 19,0MM VERMELHA",CDA,1,7.0,44.38,FERRAGEM,NaN,NaN
228,152603,,3416080,"CINTA POSTE CIRC ACO CARB 220,0MM",CDA,1,20.0,292.00,FERRAGEM,NaN,NaN
235,152603,,2402002,CARTUCHO VERMELHO FERRAMENTA AMPACT,CDA,1,8.0,24.80,FERRAGEM,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
4657,152603,,2414026,"CONETOR ATR ACO RETA 35,0/HA 16,0MM",CDA,1,3.0,10.56,FERRAGEM,NaN,NaN
4808,152603,,3300153,POSTE CONCRETO DT 12/400 44F,CDA,1,1.0,1000.50,POSTE,POSTE AT,NaN
4817,152603,,3419162,SUPORTE RETANG TRAFO 185X 95MM,CDA,1,2.0,138.76,SUCATA,NaN,NaN
4847,152603,,210497,"TRAFO 1FF 10KVA 13,8KV 220V 5T MI",CDA,1,2.0,4844.48,EQUIPAMENTO,TRAFO 2F,13.8


In [67]:
len(list(orcamento_material.groupby('NV TENSÃO').count().qnt))

1

In [68]:
####### IDENTIFICANDO FALHA NOS ORÇAMENTOS

atividades_vs_cruz = [
    ['INSTALAR EST CRUZ SIMP ISOL PINO/PILAR', 1],
    ['INSTALAR EST CRUZ SIMP ISO PINO/PILAR LV', 1],
    ['INSTALAR EST CRUZ PARA EQUIPAMENTO LV', 2],
    ['INSTALAR EST CRUZ PARA EQUIPAMENTO', 2],
    ['INSTALAR EST CRUZ DUPLA ISOL PINO/PILAR', 2],
    ['INSTALAR EST CRUZ DUPLA 1 ANCORAGEM', 2],
    ['INSTALAR EST CRUZ DUPLA 2 ANCORAGENS', 2],
    ['INST EST CRUZ DUPLA ISOL PINO/PILAR LV', 2],
    ['INST EST CRUZ DUPLA (1 ANCORAGEM) LV', 2],
    ['INST EST CRUZ DUPLA 2 ANCORAGENS LV', 2]
]
atividades_vs_cruz = pd.DataFrame(atividades_vs_cruz, columns=['descricao', 'qnt_cruz'])

projetos = orcamentos.groupby('projeto', as_index=False).sum().projeto

orcamentos_material = orcamentos.loc[orcamentos['tipo'] == 1]
orcamentos_material = orcamentos_material.merge(detalhamento_materiais[['codigo', 'CATEGORIA', 'DETALHE', 'NV TENSÃO']], on=['codigo'])
orcamentos_atividade = orcamentos.loc[orcamentos['tipo'] == 2]
orcamentos_atividade_cruzetas = orcamentos_atividade.merge(atividades_vs_cruz, on=['descricao'])
orcamentos_atividade_cruzetas['qnt_cruz_atividade'] = orcamentos_atividade_cruzetas['qnt_cruz'] * orcamentos_atividade_cruzetas['qnt']
orcamentos_atividade_cruzetas = orcamentos_atividade_cruzetas.groupby('projeto', as_index=False).sum()

diagnostico = []

for projeto in projetos:
    orcamento_atividade = orcamentos_atividade.loc[orcamentos_atividade['projeto'] == projeto]
    orcamento_material = orcamentos_material.loc[orcamentos_material['projeto'] == projeto]

    # Divergência entre materiais vs atividades orçadas [POSTES]
    poste_atividade = orcamento_atividade.loc[orcamento_atividade['descricao'] == 'INSTALAR POSTE 9 A 14 METROS'].sum().qnt + orcamento_atividade.loc[orcamento_atividade['descricao'] == 'INSTALAR POSTE 14 METROS OU SUPERIOR'].sum().qnt
    poste_material = orcamento_material.loc[orcamento_material['CATEGORIA'] == 'POSTE'].sum().qnt

    # Divergência entre materiais vs atividades orçadas [CRUZETAS]
    cruz_atividade = orcamentos_atividade_cruzetas.loc[orcamentos_atividade_cruzetas['projeto'] == projeto].sum().qnt_cruz_atividade
    cruz_material = orcamento_material.loc[orcamento_material['CATEGORIA'] == 'CRUZETA'].sum().qnt

    # Confere quantidade de estruturas bt
    qnt_postes_bt = orcamento_material.loc[orcamento_material['DETALHE'] == 'POSTE BT'].sum().qnt
    qnt_est_bt = orcamento_atividade.loc[orcamento_atividade['descricao'] == 'INSTALAR EST BT PARA COND MULTIPLEX'].sum().qnt

    if qnt_postes_bt == 0:
        existe_est_bt = False
    elif qnt_postes_bt > 0:
        existe_est_bt = True
    
    if existe_est_bt:
        if qnt_est_bt == 0:
            verificar_est_bt = 'SIM'
        if qnt_est_bt > 0:
            verificar_est_bt = 'NÃO'

    # Confere nível de tensão
    if len(list(orcamento_material.groupby('NV TENSÃO').count().qnt)) > 1:
        verificar_tensao = 'SIM'
    else:
        verificar_tensao = 'NÃO'


    diagnostico.append([projeto, poste_atividade, poste_material, cruz_atividade, cruz_material, verificar_est_bt, verificar_tensao])



diagnostico = pd.DataFrame(diagnostico, columns=['projeto', 'poste_atividade', 'poste_material', 'cruzeta_atividade', 'cruzeta_material', 'VERIFICAR EST BT', 'VERIFICAR NV TENSÃO'])

diagnostico['diferenca_poste'] = diagnostico['poste_atividade'] - diagnostico['poste_material']
diagnostico['diferenca_cruzeta'] = diagnostico['cruzeta_atividade'] - diagnostico['cruzeta_material']



C:\Users\PC1210\AppData\Local\Temp\ipykernel_20176\553432360.py:34: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  poste_material = orcamento_material.loc[orcamento_material['CATEGORIA'] == 'POSTE'].sum().qnt
C:\Users\PC1210\AppData\Local\Temp\ipykernel_20176\553432360.py:34: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  poste_material = orcamento_material.loc[orcamento_material['CATEGORIA'] == 'POSTE'].sum().qnt
C:\Users\PC1210\AppData\Local\Temp\ipykernel_20176\553432360.py:34: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid 

In [73]:
datetime.now()

datetime.datetime(2023, 3, 1, 19, 26, 40, 723806)

In [76]:
# Gera planilha final
agora = datetime.now().strftime("%Y-%m-%d %Hh%Mm%Ss")
writer = pd.ExcelWriter('Orçamentos ' + agora + '.xlsx', engine='xlsxwriter')

orcamentos.to_excel(writer, sheet_name='ORÇAMENTOS', index=False)
diagnostico.to_excel(writer, sheet_name='DIAGNÓSTICO', index=False)

writer.save()
writer.close()

C:\Users\PC1210\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
